In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import LocalOutlierFactor
from numpy import arange
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

In [2]:
benign_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample.csv')
benign_df.head(2)

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,15.854422,-0.343690,-0.086569,0.096351,0.142000,0.002723,0.706619,1.982467,1.798333,-0.034669,...,-0.059807,-0.095769,-0.109896,-0.112787,-0.020891,-0.339324,-0.282883,-0.341153,-0.315336,1
1,-0.060352,5.598377,2.253048,4.681794,3.829148,0.125465,3.569438,-0.039001,2.481753,5.125594,...,-0.059807,1.103561,0.162506,0.515708,1.673959,6.184780,12.942534,8.511326,2.479016,1


In [3]:
mssql_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\MSSQL_resample_4000.csv')
mssql_df

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.021072,-0.013958,-0.002917,-0.004205,-0.219050,-0.001653,-0.221938,-0.221586,-0.221759,-0.008236,...,0.325330,-0.001168,-0.000797,-0.001254,-0.000998,-0.012431,-0.00156,-0.012326,-0.012412,-1
1,0.021072,-0.013958,-0.002917,-0.004205,1.015610,-0.001653,1.079147,1.079875,1.079932,-0.008236,...,0.325328,-0.001168,-0.000797,-0.001254,-0.000998,-0.012431,-0.00156,-0.012326,-0.012412,-1
2,0.021072,-0.013958,-0.002917,-0.004205,-0.550300,-0.001653,-0.571010,-0.570759,-0.570994,-0.008236,...,0.325328,-0.001168,-0.000797,-0.001254,-0.000998,-0.012431,-0.00156,-0.012326,-0.012412,-1
3,0.021072,-0.013955,-0.002917,-0.004205,3.893615,-0.001653,4.111989,4.113596,4.114187,-0.008236,...,0.325328,-0.001168,-0.000797,-0.001254,-0.000998,-0.012431,-0.00156,-0.012326,-0.012412,-1
4,0.021072,-0.013958,-0.002917,-0.004205,-0.468563,-0.001653,-0.484875,-0.484600,-0.484819,-0.008236,...,0.325328,-0.001168,-0.000797,-0.001254,-0.000998,-0.012431,-0.00156,-0.012326,-0.012412,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5556,0.021072,-0.013958,-0.002917,-0.004205,-0.425544,-0.001653,-0.439541,-0.439253,-0.439464,-0.008236,...,0.325328,-0.001168,-0.000797,-0.001254,-0.000998,-0.012431,-0.00156,-0.012326,-0.012412,-1
5557,0.021072,-0.013958,-0.002917,-0.004205,-0.408336,-0.001653,-0.421408,-0.421114,-0.421322,-0.008236,...,0.325328,-0.001168,-0.000797,-0.001254,-0.000998,-0.012431,-0.00156,-0.012326,-0.012412,-1
5558,0.021072,-0.013958,-0.002917,-0.004205,-0.507281,-0.001653,-0.525676,-0.525412,-0.525638,-0.008236,...,0.325328,-0.001168,-0.000797,-0.001254,-0.000998,-0.012431,-0.00156,-0.012326,-0.012412,-1
5559,0.021072,-0.013958,-0.002917,-0.004205,-0.072784,-0.001653,-0.067803,-0.067406,-0.067552,-0.008236,...,0.325328,-0.001168,-0.000797,-0.001254,-0.000998,-0.012431,-0.00156,-0.012326,-0.012412,-1


In [4]:
benign_test_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample_4000.csv')

In [5]:
# novelty detection
df_test = pd.concat([benign_test_df, mssql_df], ignore_index=True)
df_train = benign_df

In [6]:
df_test = df_test[[' Fwd Packet Length Max', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow Packets/s', 'Bwd IAT Total', 'Fwd PSH Flags', 'Fwd Packets/s', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' min_seg_size_forward', ' Label']]
df_train = df_train[[' Fwd Packet Length Max', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow Packets/s', 'Bwd IAT Total', 'Fwd PSH Flags', 'Fwd Packets/s', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' min_seg_size_forward', ' Label']]

In [7]:
# training_data
X_train = df_train.drop([' Label'], axis=1)
y_train = df_train[' Label']
# testing_data
X_test = df_test.drop([' Label'], axis=1)
y_test = df_test[' Label']

In [8]:
for a in range(1, 3):
    for i in arange(0.1, 0.5, 0.1):
        clf = LocalOutlierFactor(n_neighbors=a, novelty=True, contamination=i)
        clf.fit(X_train)
        y_pred_test = clf.predict(X_test)
        print('Neighbour : ' + str(a))
        print('contamination : ' + str(i))
        print('---Classification Report---')
        print(classification_report(y_test, y_pred_test))

Neighbour : 1
contamination : 0.1
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      5561
           1       1.00      1.00      1.00      4003

    accuracy                           1.00      9564
   macro avg       1.00      1.00      1.00      9564
weighted avg       1.00      1.00      1.00      9564

Neighbour : 1
contamination : 0.2
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      5561
           1       1.00      1.00      1.00      4003

    accuracy                           1.00      9564
   macro avg       1.00      1.00      1.00      9564
weighted avg       1.00      1.00      1.00      9564

Neighbour : 1
contamination : 0.30000000000000004
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      5561
           1       1.00      1.00  